In [1]:
import pandas as pd
import os

def load_and_clean_data(filepath: str) -> pd.DataFrame:
    """Carga, limpia y prepara el DataFrame de avisos desde un archivo Excel."""
    try:
        df = pd.read_excel(filepath)
    except FileNotFoundError:
        print(f"¡ERROR! No se encontró el archivo con nombre: '{filepath}'")
        return None
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '', regex=False)
    date_columns = ['creado_el', 'inicio_deseado', 'fin_deseado', 'fecha_de_cierre']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    df['indicador_abc'].fillna('No Especificado', inplace=True)
    print("✅ Datos cargados y limpiados exitosamente.")
    return df

df = load_and_clean_data('avisos_prueba_tecnica.xlsx')
if df is not None:
    print("DataFrame listo para el agente.")

✅ Datos cargados y limpiados exitosamente.
DataFrame listo para el agente.


/var/folders/8p/q51pz9mn38d4_725jygnhwhh0000gn/T/ipykernel_10542/161736267.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['indicador_abc'].fillna('No Especificado', inplace=True)


In [2]:
from langchain_deepseek import ChatDeepSeek
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from dotenv import load_dotenv
import pandas as pd

# Asumiendo que 'df' ya está cargado de una celda anterior.
# Si no, incluye aquí la función load_and_clean_data y la carga del df.

# --- CORRECCIÓN AQUÍ ---
# Carga las claves del archivo .env que está en la carpeta raíz
load_dotenv('../.env')

# 1. Inicializamos el Modelo usando la API de DeepSeek
llm = ChatDeepSeek(model="deepseek-chat", temperature=0)

# 2. Creamos el Agente de Pandas
agent_executor = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True
)

# 3. Hacemos la misma pregunta para comparar resultados
pregunta = "¿Cuál es la clase de aviso más común y cuántos hay de esa clase?"
respuesta = agent_executor.invoke({"input": pregunta})

print("\n\n--- RESPUESTA FINAL (DeepSeek API) ---")
print(respuesta["output"])



> Entering new AgentExecutor chain...
Thought: I need to find the most common "clase_de_aviso" value and count how many there are of that class. I can use value_counts() on the "clase_de_aviso" column to get the frequency of each class, then find the one with the highest count.

Action: python_repl_ast
Action Input: df['clase_de_aviso'].value_counts()clase_de_aviso
N4    439
N2    104
Name: count, dtype: int64Based on the value counts, I can see that the most common "clase_de_aviso" is N4 with 439 occurrences, followed by N2 with 104 occurrences.

Final Answer: La clase de aviso más común es N4 y hay 439 avisos de esa clase.

> Finished chain.


--- RESPUESTA FINAL (DeepSeek API) ---
La clase de aviso más común es N4 y hay 439 avisos de esa clase.
